In [ ]:
import pandas as pd
import re
import yahooquery as yq
import datetime
import numpy as np

In [ ]:
# Leggi il file CSV
df = pd.read_csv('/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/tesi28.csv', sep=';')
df


## scarica i valori di borsa per ogni ticker, per ogni anno

In [ ]:
tickers = df['Ticker symbol']
valori_borsa = []
tickers_non_scaricati=[]

# Scarica i dati storici per ciascun ticker e anno
for ticker in tickers:
    yq_ticker = yq.Ticker(ticker+".mi")
    for year in range(2022, 2012, -1):
        # Specifica la data desiderata come "YYYY-MM-DD" (31 dicembre dell'anno specificato)
        datainizio = f"{year}-12-20"
        datafine = f"{year}-12-31"

        # Converti la data desiderata in un oggetto datetime
        datainizio = datetime.datetime.strptime(datainizio, "%Y-%m-%d").date()
        datafine = datetime.datetime.strptime(datafine, "%Y-%m-%d").date()


        try:
            val = yq_ticker.history(start=datainizio, end=datafine)['close'][-1]
        except Exception as e:
            print(e.__cause__)
            val = "NaN"
            tickers_non_scaricati.append(ticker)

        print(ticker, val)
        valori_borsa.append(val)



In [ ]:
print(len(valori_borsa))
print(set(tickers_non_scaricati))

## crea nuovo df
1. prendi info di interesse, moltiplica per ogni anno ogni osservazione 
2. aggiungi gli anni di interesse
3. doppio ciclo for: per ogni riga df originale prendi i primi nove valori e trasponi il risultato, passa ai prossimi nove (secondo ciclo)

#### crea df delle info 

In [ ]:
info = pd.concat([df.iloc[:, 0:5]] * 10)
info.sort_index(inplace=True)
info

#### crea colonna degli anni, aggiungi a info

In [ ]:
anni = [anno for anno in range(2022, 2012, -1)] * 28
info['Anni'] = anni
info

#### df degli indici

In [ ]:
# Rimuovi il carattere '\n' e tutti quelli che seguono dalle intestazioni delle colonne
df.columns = df.columns.str.replace(r'\n.*', '', regex=True)
nomi_colonne = df.columns.unique()[5:]
# prendi df di interesse, serve a semplificare gli indici
indici = df.iloc[:, 5:]

# crea nuovo df
new_df = pd.DataFrame()

col = 0
#per ogni set di 10 valori riga di df originario INDICI
for i in range(0, 140, 10):
    
    indice = []
    #per ogni riga AZIENDA
    for j in range(1,len(indici)+1):
        # prendi i nove valori, trasponili e aggiungili al vettore dell'indice considerato
        valori = indici.iloc[j-1:j, i:i+10].values[0]
        valori = valori.T.tolist()
        
        indice= indice + valori
        
    # aggiungi colonna a new_df con insert
    new_df.insert(col, nomi_colonne[col], indice)
    col += 1


for col in new_df.columns:
    new_df[col] = new_df[col].str.replace(',' , '.').astype(float)
    #print(type(X[col]))

new_df

#### aggiungi anche valori azionari

In [ ]:
# aggiungi i valori della borsa al df info
info['Valori Borsa'] = valori_borsa
info

##### esegui il join dei due dataframe, resetta gli indici!!


In [ ]:
#resetta indici
info = info.reset_index(drop=True)
new_df = new_df.reset_index(drop=True)
#esegui il join
dati_puliti = info.join(new_df)
dati_puliti


##### ripulisci i dati, togli le aziende che non hanno valori di brosa per almeno un anno e calcola gli indici per ogni azienda

In [ ]:
# elimina le aziende tramite una maschera
tickers_non_scaricati = list(set(tickers_non_scaricati))
maschera = dati_puliti['Ticker symbol'].isin(tickers_non_scaricati)
df_pulito = dati_puliti[~maschera]

df_pulito

In [ ]:
#calcola per ogni azienda con groupby gli indici
df_pulito['valore_2013'] = np.repeat(df_pulito[df_pulito['Anni'] == 2013]['Valori Borsa'].values, 10)
df_pulito['rapporto'] = df_pulito['Valori Borsa'] / df_pulito['valore_2013'] - 1
# decidi un threshold con cui assegnare zero e uno
treshold = 0.4
df_pulito['More Than'] = [0 if valore < treshold else 1 for valore in df_pulito['rapporto']]
df_pulito['More Than']

### salva il nuovo df 


In [ ]:
df_pulito.to_csv("dati_puliti.csv")

# CREA IL MODELLO


In [ ]:
df = pd.read_csv('/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/dati_puliti.csv', sep=',')
df

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# Separazione delle variabili di input (X) e output (y)
X = df.iloc[:, 8:-3]
y = df['More Than']

X

In [48]:

# Divisione del dataset in set di addestramento e di test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creazione e addestramento del modello di regressione logistica
model = LogisticRegression()
model.fit(X_train, y_train)

# Eseguire le previsioni sul set di test
y_pred = model.predict(X_test)

# Calcolare l'accuratezza del modello
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("accuracy", accuracy)
print("conf_matrix", conf_matrix)
print("report", report)

accuracy 0.75
conf_matrix [[13  4]
 [ 1  2]]
report               precision    recall  f1-score   support

           0       0.93      0.76      0.84        17
           1       0.33      0.67      0.44         3

    accuracy                           0.75        20
   macro avg       0.63      0.72      0.64        20
weighted avg       0.84      0.75      0.78        20



/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/env_tesi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
